In [3]:
%pip install git+https://github.com/awslabs/fast-differential-privacy.git

  Cloning https://github.com/awslabs/fast-differential-privacy.git to /private/var/folders/th/lhv5svr57tl9sd7p465ddg3h0000gn/T/pip-req-build-dnb74kyp
  Running command git clone --filter=blob:none --quiet https://github.com/awslabs/fast-differential-privacy.git /private/var/folders/th/lhv5svr57tl9sd7p465ddg3h0000gn/T/pip-req-build-dnb74kyp
  Resolved https://github.com/awslabs/fast-differential-privacy.git to commit bd81a45ae1badcdc773c5306ebfcb1fafef43966
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install transformers torch scikit-learn codecarbon


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from fastDP import PrivacyEngine
import transformers, torch
import pandas as pd

/Users/kyradresen/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/kyradresen/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Load Data

In [6]:
path="/Users/kyradresen/MasterThesis_MedTextPrivacy/data/2016r_mtsamples_final.csv"
df = pd.read_csv(path)
print(df.shape)
df.head()

(2016, 11)


,medical_specialty,transcription,age_related_sentence,extracted_text,word_count,med_masked_transcriptions,pii_masked_transcriptions,synthetic_sentence_nodp_temp0.6_topk50,synthetic_sentence_eps16_temp1_topk50,synthetic_sentence_eps8_temp1_topk100,synthetic_sentence_eps3_temp1_topk100
0,Neurology,"CC:, Confusion and slurred speech.,HX , (prima...","HX , (primarily obtained from boyfriend): This...",(primarily obtained from boyfriend): This 31 y...,43,(primarily obtained from boyfriend): This[AGE]...,(primarily obtained from boyfriend): This[AGE]...,[BOS]Neurology[SEP]The patient is a 55-year-ol...,[BOS]Neurology[SEP]The patient is a 38-year-ol...,[BOS]Neurology[SEP]Shitetron[SEPSEP]The patien...,[BOS]Neurology[SEP]The patient is a 42-year-ol...
1,Cardiovascular/Pulmonary,"PREOPERATIVE DIAGNOSES,Airway obstruction seco...","INDICATIONS FOR SURGERY,The patient is a 50-ye...",The patient is a 50-year-old white male with h...,72,The patient is a[AGE] white[SEX] with history ...,The patient is a[AGE] white[SEX] with history ...,[BOS]Cardiovascular/Pulmonary[SEP]The patient ...,[BOS]Cardiovascular/Pulmonary[SEP]The patient ...,[BOS]Cardiovascular/Pulmonary[SEP]This is a yo...,[BOS]Cardiovascular/Pulmonary[SEP]The patient ...
2,Urology,"PROCEDURE: , Elective male sterilization via b...","PROCEDURE: , Elective male sterilization via b...",Elective male sterilization via bilateral vase...,43,Elective male sterilization via bilateral vase...,Elective male sterilization via bilateral vase...,[BOS]Urology[SEP]The patient was brought to th...,[BOS]Urology[SEP]Cardiovascular/Pulmonary[SEP:...,[BOS]Urology[SEP]HISTORY OF DISHISTORY AND BEE...,[BOS]Urology[SEP]This is a 38-year-old with a ...
3,Urology,"DESCRIPTION:, The patient was placed in the s...","DESCRIPTION:, The patient was placed in the su...",The patient was placed in the supine position ...,44,The patient was placed in the supine position ...,The patient was placed in the supine position ...,[BOS]Urology[SEP]The patient is a 64-year-old ...,[BOS]Urology[SEP]The patient with a history of...,[BOS]Urology[SEP]Ophytology[SEPURE] The patien...,[BOS]Urology[SEP]The patient is referred to a ...
4,Urology,"PREOPERATIVE DIAGNOSIS: , Voluntary sterility....","INDICATIONS FOR PROCEDURE: ,A gentleman who i...",A gentleman who is here today requesting volun...,63,A gentleman who is here today requesting volun...,A gentleman who is here today requesting volun...,"[BOS]Urology[SEP]HISTORY OF PRESENT ILLNESS:, ...","[BOS]Urology[SEP]HISTORY OF THE PROCEDURE:, Th...",[BOS]Urology[SEP]The patient is a 1-year-old f...,[BOS]Urology[SEP]Neurology[SEPUE]Neuroptic[SEP...


## Finetune DistilGPT2

### Train Model

In [14]:
import torch
import torch.nn.functional as F
from torch.optim import Adam
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
from fastDP import PrivacyEngine  # Ensure fastDP is installed
from codecarbon import track_emissions


# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model and tokenizer, and move the model to the specified device
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# Define the optimizer
optimizer = Adam(model.parameters(), lr=3e-3)

# Dataset class for multi-class classification
class MedicalSpecialtyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        specialty_label = f"{self.labels[idx]}[SEP]"
        text = f"[BOS]{specialty_label}{self.texts[idx]}"

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        # The target is the same as input_ids but shifted by one position
        target_ids = input_ids.clone()
        target_ids[target_ids == self.tokenizer.pad_token_id] = -100  # Ignore padding token

        return input_ids, attention_mask, target_ids

# Load your dataframe here
# df = pd.read_csv('your_data.csv')

# Extract texts and labels from the dataframe
texts = df['extracted_text'].tolist()
labels = df['medical_specialty'].tolist()

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.05)

# Create Datasets and DataLoaders
train_dataset = MedicalSpecialtyDataset(train_texts, train_labels, tokenizer)
val_dataset = MedicalSpecialtyDataset(val_texts, val_labels, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Define the PrivacyEngine
privacy_engine = PrivacyEngine(
    model,
    batch_size=64,
    sample_size=len(train_dataset),
    epochs=5,
    target_epsilon=16,
    clipping_fn='automatic',
    clipping_mode='MixOpt',
    origin_params=None,
    clipping_style='all-layer',
)

# Attach the PrivacyEngine to the optimizer
privacy_engine.attach(optimizer)

# Training loop
@track_emissions
def training_loop():
    for epoch in range(5):
        model.train()
        for batch in train_loader:
            input_ids, attention_mask, target_ids = [x.to(device) for x in batch]

            outputs = model(input_ids, attention_mask=attention_mask, labels=target_ids)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print(f"Epoch {epoch + 1} completed.")

# Run the training loop
training_loop()

# Detach and save the privacy engine state
privacy_engine.detach()

Number of trainable components:  77 ; Number of trainable layers:  40
>>>>>>>>>>>>>>>>> Applying  automatic  per-sample gradient clipping.
>>>>>>>>>>>>>>>>> Block heads for per-sample gradient clipping are defined as: ['transformer.wte']


/home/ray/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Epoch 4 completed.
Epoch 5 completed.


## Safe Model

In [15]:
# Define the directory to save the model and tokenizer
save_directory = "./trained_model_with_classifier_distilgpt2_dpeps16_2016"

# Create the directory if it does not exist
import os
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

Model and tokenizer saved to ./trained_model_with_classifier_distilgpt2_dpeps16_2016


## Inference Model

In [16]:
# Function to generate synthetic sentences with medical specialty label
def generate_synthetic_sentence(model, tokenizer, label, max_length=80):
    model.eval()
    specialty_label = f"{label}[SEP]"
    input_ids = tokenizer.encode(f"[BOS]{specialty_label}", return_tensors='pt').to(device)
    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        no_repeat_ngram_size=4,  # Avoid repeating n-grams
        do_sample=True,  # Enable sampling
        top_k=50,  # Top-k sampling
        top_p=0.95,  # Top-p sampling (nucleus sampling)
        temperature=1.0
    )
    generated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_sentence

# Generate synthetic sentences for each row in the dataframe
synthetic_sentences = []
for index, row in df.iterrows():
    specialty = row['medical_specialty']
    synthetic_sentence = generate_synthetic_sentence(model, tokenizer, specialty, max_length=80)
    synthetic_sentences.append(synthetic_sentence)

# Append the synthetic sentences to the dataframe
df['synthetic_sentence_eps16_temp1_topk50'] = synthetic_sentences

# Save the dataframe with the synthetic sentences
df.to_csv('synthetic_sentence_eps16_temp1_topk50.csv', index=False)

print("Synthetic sentences generated and saved to synthetic_data_bossep.csv")

Synthetic sentences generated and saved to synthetic_data_bossep.csv
